In [1]:
import numpy as np
import os
from PIL import Image
from collections import defaultdict
from numpy.linalg import svd, lstsq, qr, norm
from sklearn.metrics import accuracy_score

In [7]:
def fold(matrix, shape, mode=1):
    m, n, d = shape
    q, r = matrix.shape
    tensor = np.empty(shape)
    if mode == 1:
        slice = range(0, r + 1, d)
        for i in range(n):
            b, e = slice[i], slice[i + 1]
            tensor[:, i, :] = matrix[:, b : e]
    elif mode == 2:
        slice = range(0, r + 1, m)
        for i in range(d):
            b, e = slice[i], slice[i + 1]
            tensor[:, :, i] = matrix[:, b : e].T
    else:
        slice = range(0, r + 1, n)
        for i in range(m):
            b, e = slice[i], slice[i + 1]
            tensor[i, :, :] = matrix[:, b : e].T
    
    return tensor

In [2]:
def unfold(tensor, mode=1):
    m, n, d = tensor.shape
    if mode == 1:
        return np.concatenate([tensor[:, i, :] for i in range(n)], axis=1)    
    elif mode == 2:
        return np.concatenate([tensor[:, :, i].T for i in range(d)], axis=1)    
    else:
        return np.concatenate([tensor[i, :, :].T for i in range(m)], axis=1)


In [3]:
def multiply(tensor, matrix, mode=1):
    q, r = matrix.shape
    m, n, d = tensor.shape

    if mode == 1:
        res = np.empty((q, n * d))
        res = np.concatenate([np.dot(matrix, tensor[:, i, :]) for i in range(n)], axis=1)
        return fold(res, (q, n, d), mode)
    
    elif mode == 2:
        res = np.empty((q, m * d))
        res = np.concatenate([np.dot(matrix, tensor[:, :, i].T) for i in range(d)], axis=1)
        return fold(res, (m, q, d), mode)
    
    else:
        res = np.empty((q, m * n))
        res = np.concatenate([np.dot(matrix, tensor[i, :, :].T) for i in range(m)], axis=1)
        return fold(res, (m, n, q), mode)



In [8]:
a = np.array([[-1],[2]])
b = np.array([[
    [1,2,3,4]
]])

multiply(b, a, 1)

array([[[-1., -2., -3., -4.]],

       [[ 2.,  4.,  6.,  8.]]])

In [349]:
def read_data(train_test):
    images = defaultdict(list)
    subject_name = 0
    for i in os.listdir(f'YALE_faces/{train_test}/'):
        if train_test == 'test':
            subject_name += 1
        else:
            subject_name = int(i.split('.')[0][-2:])
        image = Image.open(f'YALE_faces/{train_test}/{i}')
        image_array = np.array(image)
        image_array = image_array.flatten('F')
        images[subject_name].append(image_array)
    return images

In [238]:
def create_images_tensor(images):
    images_tensor = []
    for key, value in images.items() :
        stacked_sublist = np.stack(value, axis=1)
        images_tensor.append(stacked_sublist)
    tensor = np.stack(images_tensor, axis=0)
    return tensor
        

In [203]:
def change_shape_tensor(array):
    transposed_array = np.transpose(array, (1, 0, 2))
    reshaped_array = np.reshape(transposed_array, (array.shape[1], array.shape[0], -1))
    return np.transpose(reshaped_array, (0,2,1))

In [457]:
def HOSVD(images_tensor):
    m, n, d = images_tensor.shape
    unfold_1 = unfold(images_tensor, mode=1)
    unfold_2 = unfold(images_tensor, mode=2)
    unfold_3 = unfold(images_tensor, mode=3)

    u1, s1, v1 = svd(unfold_1, full_matrices=False)
    u2, s2, v2 = svd(unfold_2, full_matrices=False)
    u3, s3, v3 = svd(unfold_3, full_matrices=False)

    au1 = multiply(images_tensor, u1.T, mode=1)
    au2 = multiply(au1, u2.T, mode=2)
    au3 = multiply(au2, u3.T, mode=3)
    return au3, u1, u2, u3

In [495]:
images = read_data('train')
images_tensor = create_images_tensor(images)
images_tensor = change_shape_tensor(images_tensor)

images_test = read_data('test')

In [496]:
images_tensor.shape

(77760, 10, 14)

In [458]:
A, C, H, F = HOSVD(images_tensor)
A.shape

(140, 10, 14)

In [460]:
Ae = multiply (A, C, mode = 1)
Ae = multiply (Ae, H, mode = 2)

In [499]:
Ae.shape

(77760, 10, 14)

In [462]:
results = defaultdict(list)
for key, value in images_test.items():
    if key != 0:
        alpha = []
        for j in range(10):
            x, residuals, rank, singular_values = lstsq(Ae[:, j, :], np.array(value[0]), rcond=None)
            alpha.append(x)
        for i in range(14):
            results[key].append(min(norm(np.array(alpha) - F.T[:,i], axis = 1))) 

In [486]:
results_index = defaultdict(list)
for key, value in results.items():
    if min(value) <= 0.6:
        results_index[key] = value.index(min(value))
    else: 
        results_index[key] = -1

In [493]:
y_test = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
y_pred = list(results_index.values())

In [500]:
y_pred

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 -1,
 -1,
 10,
 11,
 12,
 13,
 -1,
 6,
 -1,
 6,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]

In [494]:
accuracy_score(y_test, y_pred)

0.84